# Sistema de Recomendaciones para PyMes
## Pre-procesamiento de Datos

El sistema se desarrolla con propósitos académicos para el taller ["Saturdays AI Quito"](https://quito.saturdays.ai/) 3ra edición.

Para esto, nos basamos principalmente en el siguiente conjunto de datos disponible en Kaggle: 

>**Recommender System - E-Commerce Dataset - 2020**  
Este conjunto de datos contiene 17 millones de eventos de usuarios de un sitio web de comercio electrónico del Reino Unido.
>  
Fecha de actualización del conjunto de datos: `Jan 1, 2021`  
Tamaño del archivo (comprimido): `843 MB)`  
https://www.kaggle.com/dschettler8845/recsys-2020-ecommerce-dataset

Este conjunto de datos de comercio electrónico permite crear un sistema de recomendación o una serie de otras aplicaciones sobre el mismo.

Adicionalmente, se complementa dicho conjunto de datos con información socio-demográfica tomada de la Superintendencia de Compañías del Ecuador (https://appscvs.supercias.gob.ec/rankingCias/, Ranking de Empresas 2021), en donde a cada usuario que registra una transacción, se le asigna la información de `sexo`, además de una `ciudad` del Ecuador (con su correspondiente `provincia` y `región`).

>**NOTA:** Esta asignación de información geográfica y de sexo a los usuarios del conjunto de datos original, se la realiza aleatoriamente.
>  
Esto se lo hace ya que la intención del sistema es servir de "prueba de concepto" que demostraría el tipo de predicciones que se puede hacer primeramente con las variables con las que cuenta el conjunto original de datos, y con estas variables que se le añaden.

# 1. Configuración (setup)

## 1.1 Instalar Librerías

Librería `googletrans` versión `4.0.0-rc1`  
https://pypi.org/project/googletrans/

In [ ]:
# https://pypi.org/project/googletrans/
# https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group
!pip install googletrans==4.0.0-rc1

## 1.2 Importar Librerías

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

from googletrans import Translator
from ipywidgets import widgets

In [ ]:
print("Pandas Version:", pd.__version__)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

Pandas Version: 1.1.5


## 1.3 Variables Globales

Principalmente se definen los paths de los `directorios` y nombres de `archivos` tanto de los datasets de entrada así como los que producirá el notebook.

In [ ]:
# Indica si se debe traducir [True]
# o cargar la data desde archivo [False]
TRANSLATION_OK = False

# Columnas con las Categorías del Producto y su Nombre
cols_product_name = ['cat_0',	'cat_1', 'cat_2', 'cat_3']


# Punto de montaje remoto (Google Drive)
path_drive = '/content/drive/' 

#print('os.pathsep', os.pathsep)    # Do NOT use
#print('os.path.sep', os.path.sep)  # Use this one

## PATH donde se encuentran los Datasets
path_dataset = path_drive + 'My Drive/proyecto/2. Análisis de los datos/Bases' 
print('path_dataset:', path_dataset)

# PATH dataset-1: transacciones de compra
dataset_tx = 'ds4-e_commerce/train.parquet'

# PATH dataset-2: información de clientes
dataset_cli = 'info_clientes/datos_clientes.csv'

# Diccionario con el Path de los Datasets
datasets = {
  'tx':   os.path.sep.join([path_dataset, dataset_tx]), # transacciones
  'cli': os.path.sep.join([path_dataset, dataset_cli])  # info_clientes
}


# PATH de 'Output' donde se guardarán los archivos "limpios"
#path_clean_data = 'clean'
path_clean_data = path_drive + 'My Drive/proyecto/3. Preparación de los datos'
print('path_clean_data:', path_clean_data)

# Nombre del archivo del Diccionario de Tipos de Evento (Event Type)
file_dic_event_type = 'dic_event_type.pkl'
file_dic_event_type = os.path.sep.join([path_clean_data, file_dic_event_type]) 
print('file_dic_event_type:', file_dic_event_type)

# Nombre del archivo del Diccionario de Productos Traducidos
file_dic_products = 'dic_products.pkl'
file_dic_products = os.path.sep.join([path_clean_data, file_dic_products]) 
print('file_dic_products:', file_dic_products)

# Nombre del Dataset-1: Transacciones procesadas
#file_ds_clean = 'transactions_train.parquet'
file_ds_clean = 'DataClean.parquet'
file_ds_clean = os.path.sep.join([path_clean_data, file_ds_clean]) 
print('file_ds_clean:', file_ds_clean)

# Nombre del Dataset-2: Agregación de Transacciones
file_ds_agg = 'DataClean.csv'
file_ds_agg = os.path.sep.join([path_clean_data, file_ds_agg]) 
print('file_ds_agg:', file_ds_agg)

path_dataset: /content/drive/My Drive/proyecto/2. Análisis de los datos/Bases
path_clean_data: /content/drive/My Drive/proyecto/3. Preparación de los datos
file_dic_event_type: /content/drive/My Drive/proyecto/3. Preparación de los datos/dic_event_type.pkl
file_dic_products: /content/drive/My Drive/proyecto/3. Preparación de los datos/dic_products.pkl
file_ds_clean: /content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.parquet
file_ds_agg: /content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.csv


## 1.4 Montar Google Drive

El presente notebook está pensado para ser ejecutado en [Google Colaboratory](https://colab.research.google.com) y leyendo la información desde la cuenta de [Google Drive](https://drive.google.com) del usuario.

Sin embargo si el usuario desea ejecutarlo en su equipo local, deberá modificar apropiadamente las variables definidas en la sección [1.3](https://colab.research.google.com/drive/1DhI_5EgKbrFdH2q-tcOXqMfkzb5wDz4H#scrollTo=uAsnTUZ09Sa9) (específicamente los `paths` de los directorios) y omitir la ejecución de la siguiente celda.

In [ ]:
# Cargar Datos de
# https://colab.research.google.com/notebooks/io.ipynb
from google.colab import drive

# Montar Google Drive (ingresar credenciales de: g1.recomendaciones@gmail.com)
drive.mount(path_drive)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1.5 Cargar la data de transacciones a un Pandas `Dataframe`

### 1.5.1 Verificar existencia de directorios y archivos

In [ ]:
## 1. Check Directory
print('\n---------------------------------')
print('Check the Dataset DIRECTORY:', path_dataset)
print('---------------------------------\n')
print(path_dataset)

isDir = os.path.isdir(path_dataset) 

if isDir:
  print("The DATASET directory exists")
else:
  print("The DATASET directory does not exist!!! Please verify")


## 2. Check Dataset Files

print('\n---------------------------------')
print('Check the Dataset FILES:')
print('---------------------------------')

for ds, path in datasets.items():
  print(f"\nDataset [{ds}] path: {path}")

  isFile = os.path.isfile(path) 

  if isFile:
    print("The file exists")
  else:
    print("The file does not exist!!! Please verify")


---------------------------------
Check the Dataset DIRECTORY: /content/drive/My Drive/proyecto/2. Análisis de los datos/Bases
---------------------------------

/content/drive/My Drive/proyecto/2. Análisis de los datos/Bases
The DATASET directory exists

---------------------------------
Check the Dataset FILES:
---------------------------------

Dataset [tx] path: /content/drive/My Drive/proyecto/2. Análisis de los datos/Bases/ds4-e_commerce/train.parquet
The file exists

Dataset [cli] path: /content/drive/My Drive/proyecto/2. Análisis de los datos/Bases/info_clientes/datos_clientes.csv
The file exists


### 1.5.2 Leer archivo en formato `.parquet`

Leer archivos `.parquet` en Python
https://www.youtube.com/watch?v=XFO5jdGsMek&t=124s

1. Instale el paquete `pyarrow` o `fastparquet`

  `pip install pyarrow`
  
  o 
  
  `pip install fastparquet`

>**NOTA:** No fue necesario instalar ningún paquete adicional ya que el entorno de `Google Colab` ya cuenta con un motor pre-instalado para la lectura del formato `parquet`.  

In [ ]:
#!pip list pyarrow

2. Utilice la función **`pandas.read_parquet`**

In [ ]:
# Leer dataset en formato .parquet
df_tx = pd.read_parquet(datasets['tx'], engine='auto')

In [ ]:
# Mostrar los primeros registros
df_tx.head()

,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,timestamp,ts_hour,ts_minute,ts_weekday,ts_day,ts_month,ts_year
0,2019-11-01 00:00:14 UTC,cart,1005014,samsung,503.09,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electronics,smartphone,NA,NA,2019-11-01 00:00:14,0,0,4,1,11,2019
1,2019-11-01 00:03:39 UTC,cart,1005115,apple,949.47,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electronics,smartphone,NA,NA,2019-11-01 00:03:39,0,3,4,1,11,2019
2,2019-11-01 00:05:54 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NA,NA,2019-11-01 00:05:54,0,5,4,1,11,2019
3,2019-11-01 00:07:22 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NA,NA,2019-11-01 00:07:22,0,7,4,1,11,2019
4,2019-11-01 00:10:45 UTC,cart,4804056,apple,160.57,522355747,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electronics,audio,headphone,NA,2019-11-01 00:10:45,0,10,4,1,11,2019


In [ ]:
# Obtener información del dataframe
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11495242 entries, 0 to 11495241
Data columns (total 19 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         object        
 5   user_id       object        
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  ts_hour       int16         
 14  ts_minute     int16         
 15  ts_weekday    int16         
 16  ts_day        int16         
 17  ts_month      int16         
 18  ts_year       int16         
dtypes: datetime64[ns](1), int16(6), int64(1), object(11)
memory usage: 1.2+ GB


# 2. Transformaciones al dataset

Se realizará la conversión de tipo de dato de la columna "price" y se eliminarán columnas que no se utilizarán en el análisis a realizar en el presente proyecto

## 2.1 Casting de tipo de dato del campo `"price"` a `float32`

In [ ]:
# Tranformar el campo 'price' de "string" a "float"
df_tx['price'] = df_tx['price'].astype(np.float32)

In [ ]:
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11495242 entries, 0 to 11495241
Data columns (total 19 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       object        
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  ts_hour       int16         
 14  ts_minute     int16         
 15  ts_weekday    int16         
 16  ts_day        int16         
 17  ts_month      int16         
 18  ts_year       int16         
dtypes: datetime64[ns](1), float32(1), int16(6), int64(1), object(10)
memory usage: 1.2+ GB


## 2.2 Eliminar campos

En el análisis de este proyecto no se utilizarán los siguientes campos:

- `ts_hour`       int16         
- `ts_minute`     int16         
- `ts_weekday`    int16         
- `ts_day`        int16         
- `ts_month`      int16         
- `ts_year`       int16         

In [ ]:
cols_to_delete = ['ts_hour', 'ts_minute', 'ts_weekday', 'ts_day', 'ts_month', 'ts_year'] 

df_tx.drop( cols_to_delete, axis=1, inplace=True )
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11495242 entries, 0 to 11495241
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       object        
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
dtypes: datetime64[ns](1), float32(1), int64(1), object(10)
memory usage: 1.1+ GB


# 3. Traducción de Eventos y Productos

## 3.1 Definición de Funciones y Constantes para Traducción

In [ ]:
# pandas create new column based on values from other columns / apply a function of multiple columns, row-wise
#https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o

# a) Definir función que calcula el nombre del producto
def obtener_nombre_producto(row, cols=cols_product_name):
  '''
  Recibe: rows => la fila del dataframe a evaluar
          cols => una lista ordenada con los nombres
                  de las columnas a evaluar
  
  Retorna: el nombre encontrado en la última columna
           con un valor distinto de "NA" o si el mismo
           no se ha encontrado, devuelve la palabra
           "ERROR"
  '''
  nombre = None

  # Evaluar desde la última categoria
  # empieza con 'cat_3', luego 'cat_2', etc...
  # https://stackoverflow.com/questions/869885/loop-backwards-using-indices-in-python
  for i in reversed(range(len(cols))):
    cat = cols[i]    # nombre columna, ej: cat_3
    valor = row[cat] # valor de la celda (fila, columna)

    if valor != 'NA':
      nombre = valor
      break

  # Si no se encontró ningún nombre, devolver string "ERROR"
  if nombre == None:
    nombre = "ERROR"

  return nombre

In [ ]:
# b) Función realiza la traducción de un texto, utilizando el API 'googletrans'
# NOTA: no se trata de una librería oficial de Google 
def translate(translator: Translator, text: str):
  try:
    time.sleep(0.25)
    #print(f"Translating '{text}'")
    result = translator.translate(text, src='en', dest='es').text
    return result
  except Exception as ex:
    print(f"Error when trying to translate '{text}': {ex}")
    return "ERROR"

## 3.2 Determinar el Nombre del Producto

In [ ]:
%%time
# b) Crear columna "product_name" a partir de la función "obtener_nombre_producto"
df_tx['product_name'] = df_tx.apply(lambda row: obtener_nombre_producto(row), axis=1)

CPU times: user 3min 50s, sys: 6.1 s, total: 3min 56s
Wall time: 3min 57s


In [ ]:
df_tx.head()

,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,timestamp,product_name
0,2019-11-01 00:00:14 UTC,cart,1005014,samsung,503.089996,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electronics,smartphone,NA,NA,2019-11-01 00:00:14,smartphone
1,2019-11-01 00:03:39 UTC,cart,1005115,apple,949.469971,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electronics,smartphone,NA,NA,2019-11-01 00:03:39,smartphone
2,2019-11-01 00:05:54 UTC,cart,1002542,apple,486.799988,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NA,NA,2019-11-01 00:05:54,smartphone
3,2019-11-01 00:07:22 UTC,cart,1002542,apple,486.799988,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NA,NA,2019-11-01 00:07:22,smartphone
4,2019-11-01 00:10:45 UTC,cart,4804056,apple,160.570007,522355747,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electronics,audio,headphone,NA,2019-11-01 00:10:45,headphone


## 3.3 Traducir Eventos de Compra

In [ ]:
# 1) Obtener Valores únicos del campo "event_type"
lst_event_type = df_tx['event_type'].unique()
lst_event_type

# 2) Crear manualmente diccionario EN:ES para "event_type"
# NOTA: Es posible hacerlo dado la poca cantidad de palabras
event_types = { 'cart': 'carrito', 'purchase': 'compra' }

# 3) Guardar datos en un archivo: 
with open(file_dic_event_type, 'wb') as f:
    pickle.dump(event_types, f)
    print("File '% s' saved" % file_dic_event_type)

# Print dictionary
print(event_types)

File '/content/drive/My Drive/proyecto/3. Preparación de los datos/dic_event_type.pkl' saved
{'cart': 'carrito', 'purchase': 'compra'}


## 3.4 Traducción de Categorías y Nombres de Productos

1. Obtener valores únicos de las categorías y nombres de productos

In [ ]:
# 1.1) Crear Set vacío de nombres de productos
names = set()

# 1.2) Obtener valores únicos para los campos:
# cat_0 	cat_1 	cat_2 	cat_3

for i in range( len(cols_product_name) ):
  cat = cols_product_name[i]
  #print('i:', i, '; cat:', cat)
  
  try:
    cat_values = df_tx[cat].unique()    
    #print(f'# elementos cat_{i}:', len(cat_values))
    
    cat_values = cat_values.tolist()
    #print('type(cat_values):', type(cat_values))

    # Agregar lista obtenida al set de nombres
    # names.add( cat_values ) # ERROR: unhashable type: 'list'

    # add all elements in list to the set
    # https://thispointer.com/7-ways-to-add-all-elements-of-list-to-set-in-python/
    names.update( cat_values )
  except Exception as ex:
    print(f'Error al intentar obtener datos únicos de la categoría: {i}\n{ex}')
    raise

# 1.3) Imprimir total de valores que contiene el set:
print("\n# total de nombres únicos:", len(names))


# total de nombres únicos: 161


2. Proceso de Limpieza de Datos (nombres y categorías de productos)

In [ ]:
# 2.1) Limpiar Datos:
clean_names = set()
compound_words = {}

# 2.2) Palabras compuestas
# Reemplazar "_" por espacio " "

for name in names:
  clean = name

  if (name.find("_") != -1): # Si se encontró
    # reemplazar
    clean = name.replace("_", " ")
    print(f"{name} \t\t=> {clean}")
    
    # agregar a diccionario de palabras compuestas
    compound_words[clean] = name

  # Agregar nombre limpio al set final
  clean_names.add(clean)


# 2.3) Transformar el Conjunto (set) a Lista, excluyendo el valor "NA"
clean_names = [ name for name in clean_names if name != 'NA' ]
print("# elementos en lista:", len(clean_names))

# Imprimir el número de palabras reemplazadas (palabras compuestas)
print("Nombres reemplazados:", len(compound_words))

water_heater 		=> water heater
air_conditioner 		=> air conditioner
hair_cutter 		=> hair cutter
ironing_board 		=> ironing board
anti_freeze 		=> anti freeze
coffee_grinder 		=> coffee grinder
coffee_machine 		=> coffee machine
sound_card 		=> sound card
air_heater 		=> air heater
ballet_shoes 		=> ballet shoes
power_supply 		=> power supply
country_yard 		=> country yard
sewing_machine 		=> sewing machine
living_room 		=> living room
meat_grinder 		=> meat grinder
steam_cooker 		=> steam cooker
step_ins 		=> step ins
music_tools 		=> music tools
lawn_mower 		=> lawn mower
# elementos en lista: 160
Nombres reemplazados: 19


3. Traducir nombres de productos con API `googletrans`  

  **NOTA:** La librería `googletrans 4.0.0-rc1` (que no es una librería oficial de Google) tiene problemas (y no retorna tradcción) cuando la palabra consultada tiene una traducción alternativa debido a su género.
  
  Ej: [en] trainer => [es] entrenador (masc.), entrenadora (fem.)

In [ ]:
# Utilizar API de GOOGLE para traducción
#if TRADUCIR:
# 3.1) Instanciar traductor y definir función traductora
translator = Translator()
products_error = []

# Traducir
products = dict()

for name in clean_names:
  ## KEY: original word in English 
  
  # 'get' will return either:
  # (a) the compound value if it exists (e.g. 'air_conditioner')
  #  or
  # (b) the original name (the one of the "clean name" list) if
  # it is not a compount word (i.e. if it does not exist in the
  # 'compound_words' dictionary)
  key = compound_words.get(name, name)   

  ## VALUE: Translation
  value = translate(translator, name)

  # Add an entry to the final dictionary of translated words
  products[key] = value

  # Agregar palabra a la lista de errores:
  if value == 'ERROR':
    products_error.append(key)


# Contar las palabras con ERROR: 
# 'NoneType' object is not iterable
# https://github.com/ssut/py-googletrans/issues/260

print('\n---------------------------------------------------------')
print('# de palabas traducidas:', len(products) - len(products_error))
print('# de palabas con Error al traducir: ', len(products_error))
print('---------------------------------------------------------\n')

# NOTA: La librería googletrans 4.0.0-rc1 (que no es una librería oficial
# de Google) tiene problemas (y no retorna tradcción) cuando la palabra
# consultada tiene una traducción alternativa debido a su género.
#
# Ej: [en] trainer => [es] entrenador (masc.), entrenadora (fem.)

# Imprimir diccioanrio de Productos
print(products)

Error when trying to translate 'player': 'NoneType' object is not iterable
Error when trying to translate 'kids': 'NoneType' object is not iterable
Error when trying to translate 'cooler': 'NoneType' object is not iterable
Error when trying to translate 'acoustic': 'NoneType' object is not iterable
Error when trying to translate 'peripherals': 'NoneType' object is not iterable
Error when trying to translate 'printer': 'NoneType' object is not iterable
Error when trying to translate 'jumper': 'NoneType' object is not iterable
Error when trying to translate 'fryer': 'NoneType' object is not iterable
Error when trying to translate 'mixer': 'NoneType' object is not iterable
Error when trying to translate 'monitor': 'NoneType' object is not iterable
Error when trying to translate 'cultivator': 'NoneType' object is not iterable
Error when trying to translate 'jeans': 'NoneType' object is not iterable
Error when trying to translate 'trainer': 'NoneType' object is not iterable

---------------

4. Traducir manualmente palabras no traducidas por el la librería

In [ ]:
#import ipywidgets
from ipywidgets import widgets, HBox, Label, Layout
from IPython.display import display, HTML

In [ ]:
# Diccionario de traducciones sugeridas
dic_sugerencias = {
  'acoustic': 'parlante',
  'bag': 'cartera',
  'cooler': 'enfriador',
  'cultivator': 'cultivador',
  'fryer': 'freidora',
  'jeans': 'jeans',
  'jumper': 'delantal',
  'kids': 'infantes',
  'mixer': 'batidora',
  'monitor': 'monitor',
  'peripherals': 'periféricos',
  'player': 'reproductor de sonido',
  'printer': 'impresora',
  'trainer': 'entrenador'
}

In [ ]:
# 1. Crear un componente tipo "widgets.Label" por cada palabra

# https://stackoverflow.com/questions/27114757/change-the-size-of-the-label-in-an-ipython-notebook-widget
label_layout = Layout(width='100px', height='30px')
labels = { word: widgets.Label(word, layout=label_layout) for word in products_error }

# 2. Crear diccionario de componentes tipo "widgets.Text" por cada palabra
inputs = { word: widgets.Text(value=dic_sugerencias.get(word, "") ) for word in products_error }

# 3. Crear componente Botón para Aceptar las traducciones propuestas
btn = widgets.Button(description='Aceptar')
output = widgets.Output()

# 4. Crear función de callback para el Botón
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Example
def fn_btn_aceptar(button):
  # Se asume que se asignó manualmente una
  # traducción para todas las palabras faltantes
  TRANSLATION_OK = True

  # Iterar Inputs 
  for word, component in inputs.items():
    translation = component.value
    
    #with output:
      #print(f"processing '{translation}'")

    # Validar que exista traducción
    if translation:
      # Actualizar entrada en el diccionario de palabras traducidas

      ## KEY: original word in English 
      
      # 'get' will return either:
      # (a) the compound value if it exists (e.g. 'air_conditioner')
      #  or
      # (b) the original name (the one of the "clean name" list) if
      # it is not a compount word (i.e. if it does not exist in the
      # 'compound_words' dictionary)
      key = compound_words.get(word, word)
      
      # Actualizar diccionario
      products[key] = translation

      with output:
        print(f'Diccionario actualizado: {key} -> {products[key]}')

    # Si no se ha encontrado traducción
    else:
      # Mostrar mensaje de Error
      message = f'ERROR: Hace Falta traducción para "{word}"'
      with output:
        print(message)

      # Mostrar ventana popup (alerta) con mensaje de error 
      # https://stackoverflow.com/questions/60166968/how-to-create-a-popup-in-a-widget-call-back-function-in-ipywidgets
      display(HTML("<script>alert('{}');</script>".format(message)))
  
      # Marcar 'False' a variable de Traducción
      TRANSLATION_OK = False

  # with output:
  #   print(f"[TRANSLATION_OK]: '{TRANSLATION_OK}'")


# 5. Asignar "listener" para el evento "on_click" del botón
btn.on_click(fn_btn_aceptar)

5. Obtener las Categorías y Subcategorías de los productos no traducidos  

  **NOTA:** Esto brindará un contexto a las mismas para poder realizar de mejor manera su traducción manual

In [ ]:
df_no_traducidos = df_tx.loc[ df_tx['cat_0'].isin(products_error) |
                              df_tx['cat_1'].isin(products_error) |
                              df_tx['cat_2'].isin(products_error) |
                              df_tx['cat_3'].isin(products_error) ]

# How to select unique Pandas DataFrame rows in Python
# https://www.kite.com/python/answers/how-to-select-unique-pandas-dataframe-rows-in-python
#df = df.drop_duplicates(subset = ['cat_0', 'cat_1', 'cat_2', 'cat_3'])
df_no_traducidos = df_no_traducidos[ ['brand'] + cols_product_name + ['product_name'] ].drop_duplicates(subset = cols_product_name)

# How to sort a Pandas DataFrame by multiple columns in Python
# https://www.kite.com/python/answers/how-to-sort-a-pandas-dataframe-by-multiple-columns-in-python
df_no_traducidos.sort_values(['product_name'], ascending = True)

,brand,cat_0,cat_1,cat_2,cat_3,product_name
1396,samsung,electronics,audio,acoustic,NA,acoustic
80118,logitech,computers,peripherals,camera,NA,camera
3513,lorelli,kids,carriage,NA,NA,carriage
2600,almacom,computers,components,cooler,NA,cooler
144023,huter,country_yard,cultivator,NA,NA,cultivator
6147,goo.n,kids,fmcg,diapers,NA,diapers
5313,spinmaster,kids,dolls,NA,NA,dolls
2620729,clatronic,appliances,kitchen,fryer,NA,fryer
133883,conceptclub,apparel,jeans,NA,NA,jeans
121636,didistyle,apparel,jumper,NA,NA,jumper


6. Mostrar formulario para traducir manualmente las palabras faltantes  

  **NOTAS:**  
  - El formulario se carga con la traducción sugerida por los autores
  - El proceso restante del Notebook NO podrá continuar normalmente a menos de que todas las palabras faltantes tengan una traducción.

In [ ]:
# https://stackoverflow.com/questions/38684791/changing-font-attributes-in-jupyter-notebook-label-widget
display(HTML("<style>.label { color:DarkCyan; font-weight: bold; }</style>"))

## Mostrar los componentes: 'Label' y 'Text'
for word, text_input in sorted(inputs.items()):
  # https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Label
  hbox = widgets.HBox([ labels[word].add_class('label'), text_input ])
  display( hbox )
  #display( labels[word] )
  #display( inputs[word] )

# Mostrar componente Button
display( btn, output )

Button(description='Aceptar', style=ButtonStyle())

Output()

7.  Validar que todas las palabras faltantes hayan tenido traducción

In [ ]:
fn_btn_aceptar(btn)

In [ ]:
TRANSLATION_OK = True

for word, translation in sorted(products.items()):
  if translation == 'ERROR':
    TRANSLATION_OK = False
    message = f'ERROR: Aun no existe traducción para "{word}"'
    #print(message)
    raise Exception(message)

if TRANSLATION_OK:
  print("OK: Puede proseguir normalmente con el procesamiento del resto del notebook")

OK: Puede proseguir normalmente con el procesamiento del resto del notebook


## 3.5 Guardar Diccionario de Productos traducidos

In [ ]:
if TRANSLATION_OK:

  if len(products) > 0:
    with open(file_dic_products, 'wb') as f:
      pickle.dump(products, f)
      print(f"Se ha guadado exitosamente el archivo: '{file_dic_products}'")
  else:
    print("ERROR: El diccionario de productos e encuentra vacío")

else:
  message = "ERROR: No puede efectuar esta tarea sin antes haber " \
            "traducido manualente las palabras faltantes"
  #print(message)
  raise Exception(message)

Se ha guadado exitosamente el archivo: '/content/drive/My Drive/proyecto/3. Preparación de los datos/dic_products.pkl'


In [ ]:
# 5.3 Borrar el dataframe temporal 'df_no_traducidos'
#https://www.kite.com/python/answers/how-to-clear-a-pandas-dataframe-in-python
del df_no_traducidos
df_no_traducidos = None

## 3.6 Reemplazar datos traducidos en el Dataframe de Transacciones

1. Crear función que reemplaza el contenido de una columna con la  traducción de sus celdas

In [ ]:
def translateElements(column_to_translate, dictionary):
  aux_elements = []

  # Convertir columna (Pandas.Series) hacia una lista de Python
  elements = column_to_translate.tolist()

  # Recorrer lista de celdas de la columna
  for element in elements:
    translation = 'NA'

    # si el elemento tiene un nombre
    # (es decir, es distinto de 'NA')
    if element != 'NA':
      try:
        translation = dictionary[element]
      except Exception as ex:
        translation = 'ERROR'
        #print(f'ERROR al intentar traducir [{element}]: {ex}')

    # Agregar traducción a lista de elementos traducidos
    aux_elements.append( translation )

  # Retornar lista de elementos traducidos
  return aux_elements

2. Traducir las columnas de "Tipo de Evento" y de "Categorías y Nombre de Producto"

In [ ]:
%%time
df_tx['event_type'] = translateElements(df_tx['event_type'], event_types)

df_tx['cat_0'] = translateElements(df_tx['cat_0'], products)
df_tx['cat_1'] = translateElements(df_tx['cat_1'], products)
df_tx['cat_2'] = translateElements(df_tx['cat_2'], products)
df_tx['cat_3'] = translateElements(df_tx['cat_3'], products)

df_tx['product_name'] = translateElements(df_tx['product_name'], products)

CPU times: user 15.2 s, sys: 308 ms, total: 15.5 s
Wall time: 15.5 s


3. Verificar traducciones en las primeras filas del dataframe

In [ ]:
df_tx.head()

,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,timestamp,product_name
0,2019-11-01 00:00:14 UTC,carrito,1005014,samsung,503.089996,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:00:14,teléfono inteligente
1,2019-11-01 00:03:39 UTC,carrito,1005115,apple,949.469971,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:03:39,teléfono inteligente
2,2019-11-01 00:05:54 UTC,carrito,1002542,apple,486.799988,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:05:54,teléfono inteligente
3,2019-11-01 00:07:22 UTC,carrito,1002542,apple,486.799988,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:07:22,teléfono inteligente
4,2019-11-01 00:10:45 UTC,carrito,4804056,apple,160.570007,522355747,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electrónica,audio,auricular,NA,2019-11-01 00:10:45,auricular


In [ ]:
# Verificar # de Errores en traducción
df_err = df_tx.loc[ (df_tx['cat_0'] == 'ERROR') |
                    (df_tx['cat_1'] == 'ERROR') |
                    (df_tx['cat_2'] == 'ERROR') |
                    (df_tx['cat_3'] == 'ERROR') ]

#TypeError: Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]
# https://www.programmersought.com/article/41833260226/

print("# de Errores en traducción:", len(df_err))

# de Errores en traducción: 0


4. Quitar referencias de variables a objetos utilizados en la traducción  para liberar memoria

In [ ]:
# https://stackoverflow.com/questions/23100674/python-releasing-memory-of-dictionary
names = None
clean_names = None
compound_words = None
translator = None
products_error = None
products = None
dic_sugerencias = None
labels = None
inputs = None
label_layout = None
btn = None
output = None

# 4. Integrar Dataframe de Clientes

Dado que este trabajo tiene fines académicos de aprendizaje, se integrará otro dataset con información anonimizada de Clientes, de distintas ciudades del país.

## 4.1 Asignación de información de Clientes  

1. Leer Dataset de Clientes

In [ ]:
%%time
df_cli = pd.read_csv(datasets['cli'], sep=';', encoding = "ISO-8859-1", index_col='id_usuarios')

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 2.81 s, sys: 329 ms, total: 3.14 s
Wall time: 8.71 s


In [ ]:
df_cli.head()

,REGION,PROVINCIA,CIUDAD,SEXO
id_usuarios,,,,
533326659,COSTA,EL ORO ...,MACHALA ...,F
565865924,SIERRA,AZUAY ...,CUENCA ...,F
549256216,COSTA,GUAYAS ...,GUAYAQUIL ...,F
522355747,ORIENTE,SUCUMBIOS ...,NUEVA LOJA ...,M
563558500,COSTA,GUAYAS ...,GUAYAQUIL ...,F


2. Mezclar filas del dataset de clientes

In [ ]:
# 2. mezclar filas al azar 
# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
# The idiomatic way to do this with Pandas is to use the .sample method
# of your dataframe to sample all rows without replacement:
df_cli.sample(frac=1)

,REGION,PROVINCIA,CIUDAD,SEXO
id_usuarios,,,,
525822013,SIERRA,SANTO DOMINGO DE LOS TSACHILAS ...,SANTO DOMINGO DE LOS COLORADOS ...,M
534489413,SIERRA,PICHINCHA ...,QUITO ...,F
570883267,COSTA,EL ORO ...,EL GUABO ...,M
514168740,COSTA,GUAYAS ...,GUAYAQUIL ...,M
612786945,SIERRA,PICHINCHA ...,QUITO ...,F
...,...,...,...,...
613250896,SIERRA,PICHINCHA ...,MACHACHI ...,M
525340909,COSTA,GUAYAS ...,GUAYAQUIL ...,M
557359643,COSTA,GUAYAS ...,GUAYAQUIL ...,F


3. Obtener listados únicos del `id` del **usuario** tanto del dataset _original_ (`tx`) como del _nuevo_ (`cli`)


In [ ]:
# 3.1 Determinar # de clientes y transacciones
num_clientes = len(df_cli)
num_transacciones = len(df_tx)

print(f"# clientes:\t\t\t{num_clientes:,}")
print(f"# transacciones de compra:\t{num_transacciones:,}")

print("\n------------------------------------------\n")

# 3.2 Obtener listado único de Ids de Usuarios en transacciones
lst_id_orig = df_tx['user_id'].unique() # numpy.ndarray
print(f"# usuarios (dataset original):\t\t{len(lst_id_orig):,}")

# https://www.kite.com/python/answers/how-to-return-a-column-of-a-pandas-dataframe-as-a-list-in-python
lst_id_usuarios = df_cli.index.tolist() # list
print(f"# usuarios (dataset clientes):\t\t{len(lst_id_usuarios):,}")

# 3.3 Obtener diferencia entre usuarios del ds original vs el nuevo
dif_usuarios = len(lst_id_orig) - len(lst_id_usuarios)
print(f"diferencia usuarios (tx vs. cli):\t{dif_usuarios:,}")

# clientes:			2,320,180
# transacciones de compra:	11,495,242

------------------------------------------

# usuarios (dataset original):		2,547,058
# usuarios (dataset clientes):		2,320,180
diferencia usuarios (tx vs. cli):	226,878


4. Obtener **N** usuarios con menos transacciones

  **N** = es la diferencia de usuarios entre el dataframe de Transacciones vs. el de Usuarios (variable `dif_usuarios`)

In [ ]:
# Se agrupa el dataframe por la columna 'user_id' y se utiliza
# la función "pandas.core.groupby.GroupBy.tail" para obtener
# los N (dif_usuarios) usuarios con menos transacciones.

# https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby
# pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.core.groupby.GroupBy.tail.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
# https://cmdlinetips.com/2018/02/how-to-sort-pandas-dataframe-by-columns-and-row/
df_usr_borrar = df_tx.groupby(['user_id']).size().reset_index(name='counts').sort_values('counts', ascending=False).tail(dif_usuarios)
df_usr_borrar.head()

,user_id,counts
1880863,582000832,1
1880865,582001056,1
1000379,543736591,1
1880923,582003960,1
1880906,582003034,1


5. Eliminar transacciones de los usuarios con menos compras

In [ ]:
# 5.1 Del dataframe calculado, obtener la lista de los usuarios a borrar
lst_usuarios_borrar = df_usr_borrar['user_id'].tolist()

print("type(lst_usuarios_borrar):", type(lst_usuarios_borrar))
print(f"# de Usuarios a Borrar: {len(lst_usuarios_borrar):,}")

# Determinar # de registros dataframe original antes de borrar
num_reg_df_tx = len(df_tx)
print(f"# de registros 'df_tx' [ANTES]: {num_reg_df_tx:,}")

type(lst_usuarios_borrar): <class 'list'>
# de Usuarios a Borrar: 226,878
# de registros 'df_tx' [ANTES]: 11,495,242


In [ ]:
# 5.2 Borrar aquellas transacciones cuyos usuarios pertenezcan 
# a la lista de usuarios con menos transacciones

#df.drop(df.loc[df['line_race']==0].index, inplace=True)
#df.drop(df.index[df['line_race'] == 0], inplace = True)

#df_tx.drop(df_tx.loc[df_tx['user_id'] in lst_usuarios_borrar].index, inplace=True)
#df_tx.drop(df_tx.index[df_tx['user_id'] in lst_usuarios_borrar], inplace = True)
#ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


# https://stackoverflow.com/questions/50890405/python-dataframe-deleting-rows-with-column-values-belonging-to-lists-of-values
df_tx.drop(df_tx.index[df_tx['user_id'].isin(lst_usuarios_borrar)], inplace = True)

# Determinar # de registros dataframe original antes de borrar
num_reg_df_tx = len(df_tx)
print(f"# de registros 'df_tx' [DESPUÉS]: {num_reg_df_tx:,}", )

# de registros 'df_tx' [DESPUÉS]: 11,268,364


In [ ]:
# 5.3 Actualizar lista de Usuarios de dataset original (tx)
lst_id_orig = df_tx['user_id'].unique() # numpy.ndarray
print(f"# usuarios (dataset original):\t\t{len(lst_id_orig):,}")
print(f"# usuarios (dataset clientes):\t\t{len(lst_id_usuarios):,}")

# usuarios (dataset original):		2,320,180
# usuarios (dataset clientes):		2,320,180


In [ ]:
# 5.4 Borrar el dataframe temporal 'df_usr_borrar'
#https://www.kite.com/python/answers/how-to-clear-a-pandas-dataframe-in-python
del df_usr_borrar
df_usr_borrar = None

6. Asignar al azar los **User IDs** del dataframe de clientes (`df_cli`) al campo `user_id` del dataframe de transacciones `df_tx`

In [ ]:
import random

In [ ]:
# 6.1 Definir función de asignación randómica de ID de Usuario
def asignar_user_id(id_usuario_ant, lst_ids_nuevos):
  # a) Obtener el número de elementos de la lista con los
  # IDs de usuario a utilizar
  num_elem = len(lst_ids_nuevos)

  # b) Si la lista 'lst_usuarios_new' está vacía, lanzar error
  if num_elem == 0:
    raise Exception("La lista de IDs de Clientes está vacía, ya no se pueden obtener nuevos IDs!")

  # c) Obtener randómicamente el índice de User Id nuevo a utilizar
  index = random.randint(0, num_elem - 1) # se resta 1 ya que el 2do arg. es inclusivo

  # d) Tomar y remover elemento de la lista de Ids de Usuarios a utilizar
  # de acuerdo al índice generado randómicamente
  id_nuevo = lst_ids_nuevos.pop(index)
  # NOTA: Se remueve el ID de la lista, para que el mismo
  # ya no esté disponible para futuras invocaciones de esta
  # función (futuras asignaciones de ID).

  # e) Retornar Id Nuevo
  return id_nuevo

In [ ]:
%%time
# 6.2 Crear mapa de: Id Anterior -> Id Nuevo
#dic_usuarios = { id_anterior : asignar_user_id(id_anterior, lst_id_usuarios) for id_anterior in lst_id_orig }
dic_usuarios = { id_anterior : asignar_user_id(id_anterior, lst_id_usuarios) for id_anterior in lst_id_orig }

print(f"# de IDs de Usuarios asignados: {len(dic_usuarios):,}")
print(f"# de IDs de Clientes por asignar (debería ser cero]): {len(lst_id_usuarios):,}")

# de IDs de Usuarios asignados: 2,320,180
# de IDs de Clientes por asignar (debería ser cero]): 0
CPU times: user 9min 52s, sys: 1.65 s, total: 9min 53s
Wall time: 9min 54s


In [ ]:
# 6.3 Limpiar Memoria (Referencias a objetos)
lst_id_orig = None
lst_id_usuarios = None

In [ ]:
# 6.4 Respaldar los IDs originales en otro campo
df_tx['user_id_orig'] = df_tx['user_id']

In [ ]:
%%time
# 6.5 Actualizar campo 'user_id' con el ID de dataframe de Clientes (df_cli)
#df_tx['user_id'] = df_tx.apply(lambda row: dic_usuarios[ row['user_id_orig'] ], axis=1)
#ERROR: Agota la memoria disponible y reinicia automáticamente el Kernel

# https://thispointer.com/pandas-apply-a-function-to-single-or-selected-columns-or-rows-in-dataframe/
#df_tx['user_id'] = df_tx['user_id'].apply(lambda user_id: dic_usuarios[ user_id ], axis=1)
#TypeError: <lambda>() got an unexpected keyword argument 'axis'

df_tx['user_id'] = df_tx['user_id'].apply(lambda user_id: dic_usuarios[ user_id ])

CPU times: user 11.4 s, sys: 2.03 s, total: 13.4 s
Wall time: 13.4 s


7. Asignar las columnas restantes del dataframe de clientes (`df_cli`) al de transacciones (`df_tx`)

  **NOTA:** La información estará denormalizada pero se tendrá la conveniencia de trabajar con un solo dataset, el de transacciones (`df_tx`)

In [ ]:
%%time
#REGION	PROVINCIA	CIUDAD	SEXO id_usuarios
#df_tx['region'] = df_tx['user_id'].apply(lambda user_id: df_cli.loc[df_cli['']])
#df_tx['region'] = df_tx['user_id'].apply( lambda user_id: df_cli.iloc[user_id, 0] ) # col O: REGION 

#df_tx['region'] = df_tx['user_id'].apply( lambda user_id: print(f"user_id: {user_id}"); df_cli.iloc[user_id]['REGION'] ) # col O: REGION 
#df_tx['region'] = df_tx['user_id'].apply( lambda user_id: df_cli.iloc[user_id]['REGION'] ) # col O: REGION 

# https://stackoverflow.com/questions/58572096/python-iloc-giving-indexerror-single-positional-indexer-is-out-of-bounds-in-sim
df_tx['region'] = df_tx['user_id'].apply( lambda user_id: df_cli.loc[user_id, 'REGION'] )
df_tx['provincia'] = df_tx['user_id'].apply( lambda user_id: df_cli.loc[user_id, 'PROVINCIA'] )
df_tx['ciudad'] = df_tx['user_id'].apply( lambda user_id: df_cli.loc[user_id, 'CIUDAD'] )
df_tx['sexo'] = df_tx['user_id'].apply( lambda user_id: df_cli.loc[user_id, 'SEXO'] )

CPU times: user 6min 46s, sys: 2.13 s, total: 6min 48s
Wall time: 6min 49s


8. Eliminar el campo de respaldo `user_id_orig`

In [ ]:
df_tx.drop('user_id_orig', axis=1, inplace=True)
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11268364 entries, 0 to 11495241
Data columns (total 18 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       int64         
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  product_name  object        
 14  region        object        
 15  provincia     object        
 16  ciudad        object        
 17  sexo          object        
dtypes: datetime64[ns](1), float32(1), int64(2), object(14)
memory usage: 1.6+ GB


9. Eliminar el dataframe `df_cli` para liberar memoria

In [ ]:
del df_cli
df_cli = None
dic_usuarios = None

## 4.2 Guardar dataset final de Transacciones

In [ ]:
# 1. Eliminar versión previa del archivo
if os.path.isfile(file_ds_clean):
  os.remove(file_ds_clean)
  print(f"La versión previa del archivo '{file_ds_clean}' fue eliminada!")
else:
  print(f"INFO: No se ha encontrado niguna versión previa del archivo '{file_ds_clean}'")

INFO: No se ha encontrado niguna versión previa del archivo '/content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.parquet'


In [ ]:
%%time
# 2. Guardar el Dataframe de Transacciones modificado
df_tx.to_parquet(file_ds_clean)

if os.path.isfile(file_ds_clean):
  print(f"Se ha guardado exitosamente el archivo '{file_ds_clean}'!")

Se ha guardado exitosamente el archivo '/content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.parquet'!
CPU times: user 18.8 s, sys: 3.54 s, total: 22.3 s
Wall time: 24.6 s


In [ ]:
# 3. Mostrar estuctura final del dataset de transacciones
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11268364 entries, 0 to 11495241
Data columns (total 18 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       int64         
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  product_name  object        
 14  region        object        
 15  provincia     object        
 16  ciudad        object        
 17  sexo          object        
dtypes: datetime64[ns](1), float32(1), int64(2), object(14)
memory usage: 1.6+ GB


# 5. Crear dataset de información agregada

Para facilitar la carga y uso directo de la información en herramientas de visualización (como Power BI), se agrega la información y se genera un archivo en formato `.csv` para su consumo directo.

## 5.1 Transformación y Agregación de los datos

1. Crear campo `'date'` que la información de fecha a partir de `'timestamp'`

In [ ]:
# Conservar solamente la Fecha (y no la información de Tiempo])
# https://stackoverflow.com/questions/16176996/keep-only-date-part-when-using-pandas-to-datetime
df_tx['date'] = df_tx['timestamp'].dt.date

In [ ]:
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11268364 entries, 0 to 11495241
Data columns (total 19 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       int64         
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  product_name  object        
 14  region        object        
 15  provincia     object        
 16  ciudad        object        
 17  sexo          object        
 18  date          object        
dtypes: datetime64[ns](1), float32(1), int64(2), object(15)
memory usage: 1.6+ GB


In [ ]:
df_tx.head()

,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,timestamp,product_name,region,provincia,ciudad,sexo,date
0,2019-11-01 00:00:14 UTC,carrito,1005014,samsung,503.089996,571902891,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:00:14,teléfono inteligente,SIERRA,PICHINCHA ...,QUITO ...,F,2019-11-01
1,2019-11-01 00:03:39 UTC,carrito,1005115,apple,949.469971,582574575,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:03:39,teléfono inteligente,COSTA,GUAYAS ...,GUAYAQUIL ...,F,2019-11-01
2,2019-11-01 00:05:54 UTC,carrito,1002542,apple,486.799988,524491894,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:05:54,teléfono inteligente,COSTA,MANABI ...,MONTECRISTI ...,F,2019-11-01
3,2019-11-01 00:07:22 UTC,carrito,1002542,apple,486.799988,524491894,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:07:22,teléfono inteligente,COSTA,MANABI ...,MONTECRISTI ...,F,2019-11-01
4,2019-11-01 00:10:45 UTC,carrito,4804056,apple,160.570007,591346050,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electrónica,audio,auricular,NA,2019-11-01 00:10:45,auricular,SIERRA,AZUAY ...,CUENCA ...,F,2019-11-01


2. Agregar Dataframe **(1)** agrupando por las columnas que no-numéricas y no-agregables y **(2)** sumarizar el campo "precio"

  **NOTA:** Solamente se tomarán 

In [ ]:
%%time
cols_to_group = [ 'date', 'brand', 'user_id', 'user_session', 'target', 
                  'product_id', 'cat_0', 'cat_1', 'cat_2', 'cat_3', 'product_name', 
                  'region', 'provincia', 'ciudad', 'sexo' ]

# Sumarizar el 'precio' de las columnas seleccionadas para el evento 'compra'
# https://stackoverflow.com/questions/52747805/how-to-get-rid-of-nested-column-names-in-pandas-from-group-by-aggregation
df_agg = df_tx.query("event_type == 'compra'"). \
               groupby(cols_to_group).          \
               agg({ 'price': [('price', 'sum')] })


df_agg.columns = df_agg.columns.get_level_values(1)

CPU times: user 31.9 s, sys: 2.77 s, total: 34.6 s
Wall time: 34.9 s


3. 

In [ ]:
print(f"# registros agregados: {len(df_agg ):,}")

# registros agregados: 3,545,486


# De

In [ ]:
df_agg.head()

price
date       brand user_id   user_session                         target product_id cat_0        cat_1          cat_2              cat_3 product_name       region  provincia                                          ciudad                                             sexo           
2019-10-01 NA    512432054 3f1becf8-2d11-4777-9234-eb24dda93acb 1      12300390   construcción instrumentos   taladro            NA    taladro            SIERRA  PICHINCHA                                          QUITO                                              F     25.709999
                 513434871 28080e96-d254-40f0-8cb8-b4df5a8bc192 1      6801022    ordenadores  componentes    hdd                NA    hdd                ORIENTE SUCUMBIOS                                          NUEVA LOJA                                         M     27.740000
                 513934126 07bf934b-baf2-40e1-9b06-2bb8aa5ecaee 1      12300396   construcción instrumentos   taladro            NA    taladro            COSTA   EL ORO                                             MACHALA                                            F     51.459999
                 515002498 8d57aab7-3a8e-4538-a612-a9f4f483bb67 1      6200704    accesorios   medio ambiente calentador de aire NA    calentador de aire COSTA   EL ORO                                             MACHALA                                            F     41.110001
                 515015800 a6ffb8e9-8596-4005-a35c-e6accb324b93 1      4700630    auto         accesorios     Video              NA    Video              COSTA   GUAYAS                                             GUAYAQUIL                                          M     30.889999

## 5.2 Guardar dataset en formato `.cvs`

In [ ]:
# 1. Eliminar versión previa del archivo
if os.path.isfile(file_ds_agg):
  os.remove(file_ds_agg)
  print(f"La versión previa del archivo '{file_ds_agg}' fue eliminada!")
else:
  print(f"INFO: No se ha encontrado niguna versión previa del archivo '{file_ds_agg}'")

La versión previa del archivo '/content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.csv' fue eliminada!


In [ ]:
%%time
# 2. Guardar el Dataframe de Transacciones modificado
df_agg.to_csv(file_ds_agg)

if os.path.isfile(file_ds_agg):
  print(f"Se ha guardado exitosamente el archivo '{file_ds_agg}'!")

Se ha guardado exitosamente el archivo '/content/drive/My Drive/proyecto/3. Preparación de los datos/DataClean.csv'!
CPU times: user 1min 50s, sys: 3.63 s, total: 1min 53s
Wall time: 1min 57s


# 6. Análisis Exploratorio general de los Datos

1. Verificar número de filas x columnas: `shape` property

In [ ]:
df_tx.shape

(11268364, 19)

2. Verificar Nombres de columnas y Tipos de Datos: `dtypes` property

In [ ]:
df_tx.dtypes

event_time              object
event_type              object
product_id              object
brand                   object
price                  float32
user_id                  int64
user_session            object
target                   int64
cat_0                   object
cat_1                   object
cat_2                   object
cat_3                   object
timestamp       datetime64[ns]
product_name            object
region                  object
provincia               object
ciudad                  object
sexo                    object
date                    object
dtype: object

3. Descripción Estadística: `describe()` method

In [ ]:
df_tx.describe()

,price,user_id,target
count,1.126836e+07,1.126836e+07,1.126836e+07
mean,2.861439e+02,5.568568e+08,3.651063e-01
std,3.275504e+02,3.285042e+07,4.814600e-01
min,0.000000e+00,3.948059e+07,0.000000e+00
25%,7.182000e+01,5.240465e+08,0.000000e+00
50%,1.693600e+02,5.567893e+08,0.000000e+00
75%,3.461800e+02,5.844694e+08,1.000000e+00
max,2.574070e+03,6.220901e+08,1.000000e+00


4. Contabilizar registros 'válidos': `count()` method

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.count.html

No se contabilizan los registros faltantes (`NA`)

In [ ]:
df_tx.count()

event_time      11268364
event_type      11268364
product_id      11268364
brand           11268364
price           11268364
user_id         11268364
user_session    11268308
target          11268364
cat_0           11268364
cat_1           11268364
cat_2           11268364
cat_3           11268364
timestamp       11268364
product_name    11268364
region          11268364
provincia       11268364
ciudad          11268364
sexo            11268364
date            11268364
dtype: int64

5. Contabilizar registros NA: `isna()` method

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html

A manera de "negativo" de la anterior función (`count()`), la función **`isna()`** nos indica el número valores faltantes (`NA`).

Para esto, es útil el uso de la función agrupadora **`aggregate`** junto con la expresión lambda abajo indicada, que sumariza `sum()` los registros faltantes, identificados por `isna()`:

In [ ]:
df_tx.isna().sum()

event_time       0
event_type       0
product_id       0
brand            0
price            0
user_id          0
user_session    56
target           0
cat_0            0
cat_1            0
cat_2            0
cat_3            0
timestamp        0
product_name     0
region           0
provincia        0
ciudad           0
sexo             0
date             0
dtype: int64

6. Información del Dataframe

In [ ]:
df_tx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11268364 entries, 0 to 11495241
Data columns (total 19 columns):
 #   Column        Dtype         
---  ------        -----         
 0   event_time    object        
 1   event_type    object        
 2   product_id    object        
 3   brand         object        
 4   price         float32       
 5   user_id       int64         
 6   user_session  object        
 7   target        int64         
 8   cat_0         object        
 9   cat_1         object        
 10  cat_2         object        
 11  cat_3         object        
 12  timestamp     datetime64[ns]
 13  product_name  object        
 14  region        object        
 15  provincia     object        
 16  ciudad        object        
 17  sexo          object        
 18  date          object        
dtypes: datetime64[ns](1), float32(1), int64(2), object(15)
memory usage: 1.6+ GB


1. El dataset tiene las siguientes variables:

|Variable|Tipo de Dato|Descripción|
|-|-|-|
|**InvoiceNo**|       object|Número de Factura|
|**StockCode**|       object|Código el producto|
|**Description**|          object|Descripción del producto|
|**Quantity**|       int64|Cantidad del Producto correspondiente a la transacción|
|**UnitPrice**|        float64|Precio Unitario del Producto|
|**CustomerID**|        float64|ID del Cliente|
|**Country**|object|País desde donde realizan la compra|

2. El tipo de variables 

* Numéricas (Cuantitativas)
  * Discretas

  * Contínuas

* Categórica (No Cuantitativa)


In [ ]:
df_tx.head()

,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,timestamp,product_name,region,provincia,ciudad,sexo,date
0,2019-11-01 00:00:14 UTC,carrito,1005014,samsung,503.089996,571902891,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:00:14,teléfono inteligente,SIERRA,PICHINCHA ...,QUITO ...,F,2019-11-01
1,2019-11-01 00:03:39 UTC,carrito,1005115,apple,949.469971,582574575,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:03:39,teléfono inteligente,COSTA,GUAYAS ...,GUAYAQUIL ...,F,2019-11-01
2,2019-11-01 00:05:54 UTC,carrito,1002542,apple,486.799988,524491894,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:05:54,teléfono inteligente,COSTA,MANABI ...,MONTECRISTI ...,F,2019-11-01
3,2019-11-01 00:07:22 UTC,carrito,1002542,apple,486.799988,524491894,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electrónica,teléfono inteligente,NA,NA,2019-11-01 00:07:22,teléfono inteligente,COSTA,MANABI ...,MONTECRISTI ...,F,2019-11-01
4,2019-11-01 00:10:45 UTC,carrito,4804056,apple,160.570007,591346050,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electrónica,audio,auricular,NA,2019-11-01 00:10:45,auricular,SIERRA,AZUAY ...,CUENCA ...,F,2019-11-01
